# Self-Guessing Hybrid Search (aka Part 3/3)

Assume we have a working recipe for hybrid search in the form `(query, [keyword]) => [(snippet, siilarity)]`.

Now the goal of this notebook is, _can we self-guess the keywords from the query and have the caller supply just the query?_

In [9]:
import os
from functools import partial
from dotenv import load_dotenv

_ = load_dotenv('.env')

In [2]:
import cassio

In [3]:
cassio.init(
    token=os.environ['ASTRA_DB_APPLICATION_TOKEN'],
    database_id=os.environ['ASTRA_DB_ID'],
    keyspace=os.environ.get('ASTRA_DB_KEYSPACE'),
)
session = cassio.config.resolve_session()
keyspace = cassio.config.resolve_keyspace()

In [4]:
import openai

embedding_model_name = "text-embedding-ada-002"

def get_embeddings(texts):
    result = openai.Embedding.create(
        input=texts,
        engine=embedding_model_name,
    )
    return [res.embedding for res in result.data]

### Recap

Let's use the latest "hybrid search" function from the previous investigation. One important point is that we'll proceed with the keywords in OR (i.e. a single keyword match suffices for a hit). This enables a meaningful contribution to the "score" from the keyword side; but most important, with the keywords being self-guessed, it protects somewhat from too-demanding "guesses" from the query.

**We have packaged the latest machinery from Part 2 into a Python module to reduce clutter, nothing new**

_(We just renamed the final hybrid-search function for convenience and made sure the DB parameters pass through the calls)_

In [6]:
from kw_hybrid_tools import hybrid_search_with_kw, show

Let's define a handy shortcut and run a sanity check (to be compared with the "QUERY7/KW7" run in the previous notebook):

In [11]:
hybrid_kw = partial(hybrid_search_with_kw, session=session, keyspace=keyspace, get_embeddings=get_embeddings)

KW0 = ['support', 'chat']
QUERY0 = "How come I cannot chat?"

print(f"[with safe prefetch] QUERY: '{QUERY0}', KEYWORDS: \'{', '.join(KW0)}\'")
show(hybrid_kw(query=QUERY0, keywords=KW0))

[with safe prefetch] QUERY: 'How come I cannot chat?', KEYWORDS: 'support, chat'
    [1] 0.96758 "I cannot open the support chat."
    [2] 0.96159 "I see no messages in the support chat."
    [3] 0.95498 "The support chat on the website is lagging."


## Guessing the keywords from the query